In [25]:
from datetime import datetime
from multiprocessing import Pool

import time
import os
import sys
from pathlib import Path

import numpy as np
import matplotlib
matplotlib.use('nbagg')
#from matplotlib import style
#style.use('ggplot')
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

import astropy.units as u
from astropy import stats
from astropy.io import fits
from mmtwfs.wfs import *
from mmtwfs.zernike import ZernikeVector
from mmtwfs.telescope import MMT

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# instantiate all of the WFS systems...
wfs_keys = ['newf9', 'f9', 'f5', 'mmirs']
wfs_systems = {}
wfs_names = {}
for w in wfs_keys:
    wfs_systems[w] = WFSFactory(wfs=w)
    wfs_names[w] = wfs_systems[w].name

In [48]:
def check_wfs(p):
    """
    Check to see which system was used on a given night
    """
    if Path.exists(p / "F9"):  # wow, i really like pathlib.Path...
        return "f9"
    elif Path.exists(p / "F5"):
        return "f5"
    elif Path.exists(p / "MMIRS"):
        return 'mmirs'
    else:
        return None

def process_image(f):
    """
    Process FITS file, f, to get info we want from the header and then analyse it with the 
    appropriate WFS instance. Return results in a comma-separated line that will be collected 
    and saved in a CSV file.
    """
    if "ave" not in f.name:
        with fits.open(f) as h:
            hdr = h[0].header
            if 'AIRMASS' in hdr:
                airmass = hdr['AIRMASS']
            else:
                airmass = np.nan
            # we need to fix the headers in all cases to have a proper DATE-OBS entry with
            # properly formatted FITS timestamp.  in the meantime, this hack gets us what we need 
            # for analysis in pandas.
            dateobs = hdr['DATEOBS']
            ut = hdr['ut']
            timestring = dateobs + " " + ut + " UTC"
            dtime = datetime.strptime(timestring , "%a %b %d %Y %H:%M:%S %Z")
            obstime = dtime.isoformat()
            # being conservative here and only using data that has proper slope determination
            # and wavefront solution. also want to get statistics on the quality of the wavefront fits.
            results = wfs_systems[wfskey].measure_slopes(f, plot=False)
            if results['slopes'] is not None:
                zresults = wfs_systems[wfskey].fit_wavefront(results, plot=False)
                line = "%s,%s,%s,%f,%f,%f,%f,%f\n" % (
                    obstime,
                    wfskey,
                    f.name, 
                    airmass,
                    results['seeing'].value,
                    results['raw_seeing'].value,
                    results['fwhm'],
                    zresults['residual_rms'].value
                )
                return line
            else:
                return None
    return None

In [50]:
rootdir = Path("/Users/tim/MMT/wfsdat/test")
dirs = rootdir.glob("*")  # pathlib, where have you been all my life!
csv_header = "time,wfs,file,airmass,seeing,raw seeing,fwhm,wavefront rms\n"
for d in dirs:
    if d.is_dir():
        if Path.exists(d / "seeing_results.csv"):
            print("Already processed %s..." % d.name)
        else:
            try:
                lines = []
                lines.append(csv_header)
                night = int(d.name)  # valid WFS directories are ints of the form YYYYMMDD. if not this form, int barfs
                msg = "checking %d... " % night
                wfskey = check_wfs(d)
                if wfskey is not None:
                    if wfskey == "mmirs":
                        rawd = d / "rawdata"
                        fitsfiles = rawd.glob("*.fits")
                    else:
                        fitsfiles = d.glob("*.fits")
                    if wfskey == "f9" and night > 20170510:
                        wfskey = "newf9"
                    msg += " using %s." % wfskey
                    print(msg)
                    with Pool(processes=8) as pool:  # my mac's i7 has 4 cores + hyperthreading so 8 virtual cores. 
                        plines = pool.map(process_image, fitsfiles)  # plines comes out in same order as fitslines!
                    plines = list(filter(None.__ne__, plines))  # trim out any None entries
                    lines.extend(plines)
                    with open(d / "seeing_results.csv", "w") as f:
                        f.writelines(lines)
                else:
                    msg = "No valid wfskey for %s..." % d
                    print(msg)
            except ValueError:  # this means running int(d.name) failed so it's not a valid directory...
                print("Skipping %s..." % d.name)

Already processed 20170505...


In [ ]:
t = "Thu May 18 2017"
datetime.strptime(t, "%a %b %d %Y")

In [19]:
with open("test.csv", "w") as f:
    f.writelines(lines)

In [21]:
!more test.csv

time, wfs, file, airmass, seeing, raw seeing, fwhm, wavefront rms
2017-05-05T03:05:27, f5, auto_wfs_0000.fits, 1.330000, 1.088146, 1.291213, 8.005576, 1482.095233 
2017-05-05T03:05:55, f5, auto_wfs_0001.fits, 1.330000, 1.013141, 1.202212, 7.581082, 1195.299068 
2017-05-05T03:06:11, f5, auto_wfs_0002.fits, 1.330000, 0.968784, 1.149576, 7.328668, 1206.347418 
2017-05-05T03:06:26, f5, auto_wfs_0003.fits, 1.330000, 0.861292, 1.022025, 6.712848, 1274.501378 
2017-05-05T03:07:18, f5, auto_wfs_0004.fits, 1.320000, 0.865118, 1.021927, 6.712375, 1409.837661 
2017-05-05T03:07:33, f5, auto_wfs_0005.fits, 1.320000, 0.872814, 1.031018, 6.756455, 1398.160690 
2017-05-05T03:07:50, f5, auto_wfs_0006.fits, 1.320000, 1.114053, 1.315982, 8.123193, 1634.588775 
2017-05-05T03:08:38, f5, auto_wfs_0007.fits, 1.320000, 0.998931, 1.179994, 7.474658, 266.336654 
2017-05-05T03:08:54, f5, auto_wfs_0008.fits, 1.320000, 1.034293, 1.221765, 7.674592, 273.137820 
2017-05-05T03:09:10, f5, auto_wfs_0009.fits, 1.310000,

In [41]:
d / "seeing.csv"

PosixPath('/Users/tim/MMT/wfsdat/test/20170505/seeing.csv')